In [1]:
import os
import mne  # For EEG/MEG data processing
from mne.preprocessing import ICA  # For Independent Component Analysis
from mne_icalabel import label_components  # For automatic ICA component labeling
from autoreject import AutoReject  # For automatic artifact rejection
import numpy as np  # For numerical operations
from autoreject import get_rejection_threshold  # For getting rejection thresholds
import gc  # Import the garbage collector interface for memory management

In [2]:
## Subjects and Sessions to include
EEG_INFO = {
    'SUBJ_NUMS': ['10002', '10003', '10004', '10008', '10011', '10012', '10016', '10019', 
                  '10021', '10022', '10023', '10024', '10028', '10031', '10032', '10034', 
                  '10036', '10037', '10039', '10040', '10046', '10048', '10056', '10058'],
    
    'SESS_NUMS' : ['W00', 'W00_2', 'W00_3', 'W00_4',
                    'W08', 'W08_2', 'W08_3', 'W08_4',
                    'W12', 'W12_2', 'W12_3', 'W12_4',
                    'W20', 'W20_2', 'W20_3', 'W20_4'],
}
SUBJ_NUMS = EEG_INFO['SUBJ_NUMS']
SESS_NUMS = EEG_INFO['SESS_NUMS']

In [3]:
############# Directory paths
raw_eeg_path = '01_Raw_EEG'
epoch_ham_save_path = '02_Epoch_EEG/Concatenated'

In [4]:
############# General data settings

# Specify EEG signal epoch length (seconds)
TMIN = 0  # Start time of the epoch
TMAX = 10  # End time of the epoch

# Initialize ICA (Independent Component Analysis) object
# We will use the 'extended infomax' method for fitting the ICA, to comply with
# the ICLabel requirements.
ica = ICA(
    n_components=0.999999,  # Retain 99.9999% of the variance
    max_iter="auto",  # Automatically determine the number of iterations
    method="infomax",  # Use the Infomax ICA algorithm
    random_state=97,  # Set random seed for reproducibility
    fit_params=dict(extended=True),  # Use extended Infomax algorithm
)

# Define EEG channel names
REF_CHS = ['A1', 'A2']  # Reference channels (ear clips)

# List of EEG channel names following the 10-20 system
CHAN_NAMES = ['F7', 'Fp1', 'Fp2', 'F8', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'P8', 
              'P7', 'Pz', 'P4', 'T3', 'P3', 'O1', 'O2', 'C4', 'T4']

In [5]:
def apply_hamming(original_sig, fs, len_hamm):
    """
    Apply a Hamming window taper to the beginning and end of a signal.
    
    Parameters:
    original_sig : array-like
        The original signal to which the Hamming window will be applied.
    fs : float
        The sampling frequency of the signal.
    len_hamm : float
        The length of the Hamming window taper in seconds.
    
    Returns:
    new_sig : array-like
        The signal with the Hamming window taper applied.
    """
    
    # Calculate the number of samples for the Hamming taper
    ind_hamm = int(fs * len_hamm)
    
    # Create an array of ones the same size as the original signal
    window_sig = np.ones_like(original_sig)
    
    # Create a Hamming window twice the length of the taper
    hamming_window = np.hamming(len_hamm * 2 * fs)
    
    # Apply the first half of the Hamming window to the beginning of the signal
    window_sig[..., :ind_hamm] = hamming_window[:ind_hamm]
    
    # Apply the second half of the Hamming window to the end of the signal
    window_sig[..., -ind_hamm:] = hamming_window[-ind_hamm:]
    
    # Multiply the original signal by the windowing function
    new_sig = original_sig * window_sig
    
    return new_sig

In [6]:
## Loop through each session file
for sub_ind, sub in enumerate(SUBJ_NUMS):  # For each subject
    for sess_ind, sess in enumerate(SESS_NUMS):  # For each session
        try:
            ###############################################################################
            # AutoReject
            ###############################################################################
            # Name of eeg session
            full_sess_name = str(sub) + "_" + str(sess)
            print('\n\nRUNNING SUBJECT:  ' + str(sub) + '_' + str(sess) + '\n\n')
            subj_fname = full_sess_name + ".vhdr"
            full_path = os.path.join(raw_eeg_path, subj_fname)

            # Check if the file exists
            if not os.path.exists(full_path):
                print('Current Subject ' + str(sub) + '_' + str(sess) + ' does not exist')
                continue

            # Load EEG (.vhdr) file
            eeg_data = mne.io.read_raw_brainvision(full_path, preload=True)

            # Check and update ExG channel types
            exg_channels = ['ExG 1', 'ExG 2', 'ExG 3', 'ExG 4']
            existing_exg_channels = [ch for ch in exg_channels if ch in eeg_data.ch_names]
            if existing_exg_channels:
                new_channel_types = {ch: 'eog' for ch in existing_exg_channels}
                eeg_data.set_channel_types(new_channel_types)
                print(f"Updated channel types for: {', '.join(existing_exg_channels)}")
            else:
                print("No ExG channels found to update.")

            # Create montage for optional scalp electrode mapping
            montage = mne.channels.make_standard_montage('standard_1020', head_size='auto')
            eeg_data.set_montage(montage)

            # Set EEG references
            eeg_ix = mne.pick_channels(eeg_data.ch_names, include=CHAN_NAMES)
            groups = dict(A1=eeg_ix)
            eeg_combined = mne.channels.combine_channels(eeg_data, groups, method="mean")
            print(eeg_combined.info["ch_names"])
            eeg_data = eeg_data.add_channels([eeg_combined])
            eeg_data.set_eeg_reference(ref_channels=REF_CHS)

            ###############################################################################
            # Calculate Global Reject Threshold
            ###############################################################################
            # Create events and epochs for rejection threshold calculation
            events = mne.make_fixed_length_events(eeg_data.pick(CHAN_NAMES), start=0, stop=eeg_data.times[-1])
            complete_epoch = mne.Epochs(eeg_data, events=events, baseline=None, preload=True)
            reject = get_rejection_threshold(complete_epoch, decim=1)
            print('The rejection dictionary is %s' % reject)

            ###############################################################################
            # ICA
            ###############################################################################
            # High-pass filter data for running ICA
            eeg_data.filter(l_freq=1, h_freq=100, fir_design='firwin')
            FS = eeg_data.info['sfreq']  # Sampling frequency of signals

            # Fit ICA
            ica.fit(eeg_data, reject=reject)

            # Apply automatic ICA component labeling
            ic_labels = label_components(eeg_data, ica, method="iclabel")
            labels = ic_labels["labels"]
            exclude_idx = [idx for idx, label in enumerate(labels) if label not in ["brain", "other"]]

            # Apply ICA
            raw_ica = eeg_data.copy()
            ica.apply(raw_ica, exclude=exclude_idx)

            ###############################################################################
            # AutoReject (Local)
            ###############################################################################
            # Create events and epochs for AutoReject
            events = mne.make_fixed_length_events(raw_ica.pick(CHAN_NAMES), start=0, stop=eeg_data.times[-1], duration=TMAX)
            epochs_ar_ica = mne.Epochs(raw_ica, events=events, tmin=TMIN, tmax=TMAX, baseline=None, preload=True)

            # Run AutoReject
            ar = AutoReject(n_jobs=4, random_state=42, verbose=False, picks='eeg')
            epochs_ar_final, reject_log = ar.fit_transform(epochs_ar_ica, True)

            ###############################################################################
            # Concatenate and apply Hamming window
            ###############################################################################
            # Apply Hamming window to each epoch
            windowed_data = np.array([apply_hamming(epoch, FS, len_hamm=0.1) for epoch in epochs_ar_final.get_data()])
            combined_data = np.concatenate(windowed_data, axis=-1)

            # Create a new Raw object from the combined data
            info = epochs_ar_final.info
            raw_combined = mne.io.RawArray(combined_data, info)
            
            # Create a new Epochs object with the windowed data
            new_events = mne.make_fixed_length_events(raw_combined, start=0, stop=raw_combined.times[-1], duration=TMAX)
            windowed_epochs = mne.Epochs(raw_combined, events=new_events, tmin=TMIN, tmax=TMAX, baseline=None, preload=True)

            # Convert epoch data into one single epoch
            data = windowed_epochs.get_data()
            combined_data_single = np.concatenate(data, axis=-1)

            # Convert back to epoch format
            concatenated_data = combined_data_single[None, :, :]  # Add an epoch dimension
            info = epochs_ar_ica.info.copy()
            concatenated_epoch = mne.EpochsArray(concatenated_data, info)

            # Save epoch object to disk
            concatenated_epoch.save(epoch_ham_save_path + '/' + full_sess_name + '_epo.fif', overwrite=True)
            print('\n\nSAVED:  ' + str(sub) + '_' + str(sess) + '\n\n')

            # Clear unnecessary variables
            del epochs_ar_final, concatenated_epoch, combined_data, data, epochs_ar_ica, raw_ica, eeg_data, windowed_epochs
            gc.collect()  # Explicitly call garbage collector

        except Exception as e:
            print(f"An error occurred for subject {sub} session {sess}: {e}")
            continue



RUNNING SUBJECT:  10002_W00


Extracting parameters from I:/CBD_ASD_EEG/01_Raw_EEG\10002_W00.vhdr...
Setting channel info structure...
Reading 0 ... 52693  =      0.000 ...   105.386 secs...
No ExG channels found to update.
Creating RawArray with float64 data, n_channels=1, n_times=52694
    Range : 0 ... 52693 =      0.000 ...   105.386 secs
Ready.
['A1']
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
105 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 105 events and 351 original time points ...
1 bad epochs dropped
Estimating rejection dictionary for eeg
The rejection dictionary is {'eeg': 0.040117784819908825}
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 6.9s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying ICA to Raw instance
    Transforming to ICA space (19 components)
    Zeroing out 0 ICA components
    Projecting back using 19 PCA components
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 10 events and 5001 original time points ...
0 bad epochs dropped


C:\Users\Christian\PycharmProjects\test\Lib\site-packages\mne\bem.py:1134: RuntimeWarning: Mean of empty slice.
  radius_init = radii.mean()
C:\Users\Christian\PycharmProjects\test\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\Christian\PycharmProjects\test\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Christian\PycharmProjects\test\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Dropped 7 epochs: 0, 2, 3, 4, 5, 7, 9
Creating RawArray with float64 data, n_channels=19, n_times=15003
    Range : 0 ... 15002 =      0.000 ...    30.004 secs
Ready.
30.004
[[    0     0     1]
 [ 5000     0     1]
 [10000     0     1]]
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 3 events and 5001 original time points ...
0 bad epochs dropped
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated


SAVED:  10002_W00




RUNNING SUBJECT:  10002_W00_2


Current Subject 10002_W00_2 does not exist


RUNNING SUBJECT:  10002_W00_3


Current Subject 10002_W00_3 does not exist


RUNNING SUBJECT:  10002_W00_4


Current Subject 10002_W00_4 does not exist


RUNNING SUBJECT:  10002_W08


Extracting parameters from I:/CBD_ASD_EEG/01_Raw_EEG\10002_W08.vhdr...
Setting channel info structure...
Reading 0 ... 294112  =      0.000 ...   588.224 secs...


...

    Rejecting  epoch based on EEG : ['F7', 'Fp2', 'P7']
Artifact detected in [190000, 191000]
    Rejecting  epoch based on EEG : ['Cz', 'P7', 'Pz', 'T3']
Artifact detected in [191000, 192000]
    Rejecting  epoch based on EEG : ['Fp2']
Artifact detected in [193000, 194000]
    Rejecting  epoch based on EEG : ['F7', 'Fp1', 'Fp2', 'F8', 'Fz', 'P8', 'P7', 'Pz', 'P4', 'T3', 'C4']
Artifact detected in [197000, 198000]
    Rejecting  epoch based on EEG : ['F7', 'Fp2', 'P7']
Artifact detected in [198000, 199000]
    Rejecting  epoch based on EEG : ['F7', 'Fp2']
Artifact detected in [199000, 200000]
    Rejecting  epoch based on EEG : ['F7', 'Fp2', 'F8', 'T3']
Artifact detected in [200000, 201000]
    Rejecting  epoch based on EEG : ['F7', 'F8', 'T3']
Artifact detected in [201000, 202000]
    Rejecting  epoch based on EEG : ['F7']
Artifact detected in [206000, 207000]
    Rejecting  epoch based on EEG : ['F8', 'P7', 'T3']
Artifact detected in [209000, 210000]
    Rejecting  epoch based on EEG

C:\Users\Christian\PycharmProjects\test\Lib\site-packages\mne\bem.py:1134: RuntimeWarning: Mean of empty slice.
  radius_init = radii.mean()
C:\Users\Christian\PycharmProjects\test\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\Christian\PycharmProjects\test\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Christian\PycharmProjects\test\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Dropped 27 epochs: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16, 31, 32, 43, 45, 46, 47, 48, 49, 50, 53, 55, 56, 57
Creating RawArray with float64 data, n_channels=19, n_times=155031
    Range : 0 ... 155030 =      0.000 ...   310.060 secs
Ready.
310.06
[[     0      0      1]
 [  5000      0      1]
 [ 10000      0      1]
 [ 15000      0      1]
 [ 20000      0      1]
 [ 25000      0      1]
 [ 30000      0      1]
 [ 35000      0      1]
 [ 40000      0      1]
 [ 45000      0      1]
 [ 50000      0      1]
 [ 55000      0      1]
 [ 60000      0      1]
 [ 65000      0      1]
 [ 70000      0      1]
 [ 75000      0      1]
 [ 80000      0      1]
 [ 85000      0      1]
 [ 90000      0      1]
 [ 95000      0      1]
 [100000      0      1]
 [105000      0      1]
 [110000      0      1]
 [115000      0      1]
 [120000      0      1]
 [125000      0      1]
 [130000      0      1]
 [135000      0      1]
 [140000      0      1]
 [145000      0      1]
 [150000      0      1]]
N

C:\Users\Christian\AppData\Local\Temp\ipykernel_21924\2183489275.py:105: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  windowed_data = np.array([apply_hamming(epoch, FS, len_hamm=0.1) for epoch in epochs_ar_final.get_data()])
C:\Users\Christian\AppData\Local\Temp\ipykernel_21924\2183489275.py:121: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = windowed_epochs.get_data()




SAVED:  10058_W20




RUNNING SUBJECT:  10058_W20_2


Current Subject 10058_W20_2 does not exist


RUNNING SUBJECT:  10058_W20_3


Current Subject 10058_W20_3 does not exist


RUNNING SUBJECT:  10058_W20_4


Current Subject 10058_W20_4 does not exist
